<a href="https://colab.research.google.com/github/AntonSangho/SmartFarm_lecture/blob/main/SmartFarm_lecture/src/simpletest/NDVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 이미지 업로드 및 전처리

식물이 빛과 상호작용하는 방식은 매우 특별합니다. 햇빛을 보면 우리 눈에는 하얀색으로 보이지만, 사실 여러 가지 색깔의 빛이 섞여 있습니다. 무지개를 보면 이 빛들이 각각의 색으로 나뉘어 보이죠.

식물은 이 중에서 특정한 빛만 선택적으로 사용합니다. 예를 들어, 우리가 먹는 채소의 녹색 잎을 보면:
- **빨간색 빛**은 광합성에 사용
- **초록색 빛**은 반사 (그래서 우리 눈에 초록색으로 보임)
- **적외선**은 대부분 반사 (잎이 뜨거워지는 것을 방지)

마치 우리가 뜨거운 여름날 흰색 옷을 입어 열을 반사하는 것처럼, 식물도 건강할수록 적외선을 더 잘 반사합니다.

라즈베리파이 NoIR 카메라는 이런 식물의 특성을 이용해서 식물의 건강 상태를 촬영할 수 있습니다. 일반 카메라는 적외선을 차단하는 필터가 있지만, NoIR 카메라는 이 필터를 제거하여 식물이 반사하는 적외선까지 촬영할 수 있습니다.

**샘플 이미지 다운로드**: https://drive.google.com/file/d/13BKvqcuhnI5gNVK9ij4G1Sv3GT03zbN6/view?usp=drive_link

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io
from PIL import Image

def load_and_preprocess_image():
    """이미지 업로드 및 전처리 함수"""
    print("식물 이미지를 업로드하세요.")
    uploaded = files.upload()
    img_name = list(uploaded.keys())[0]

    # 이미지 로드 및 RGB 변환
    img = cv2.cvtColor(cv2.imread(img_name), cv2.COLOR_BGR2RGB)

    # 원본 이미지 시각화
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.title('Original Plant Image')
    plt.axis('off')
    plt.show()

    return img

# 이미지 로드 및 전처리
original_img = load_and_preprocess_image()

print("\n=== 이미지 정보 ===")
print(f"이미지 크기: {original_img.shape[0]} x {original_img.shape[1]} 픽셀")
print(f"채널 수: {original_img.shape[2]}")

# 2. 대비 향상 (Contrast Enhancement)

사진을 더 선명하게 만들어보겠습니다. 흐린 날 찍은 사진을 선명하게 보정하는 것처럼, 우리도 식물의 상태를 더 잘 볼 수 있도록 이미지를 개선할 수 있습니다.

예를 들어, 흐린 날 찍은 풍경사진을 보면:
- 하늘과 나무가 뿌옇게 보임
- 색상이 선명하지 않음
- 나무의 잎사귀 구분이 어려움

이미지의 대비를 높이면:
- 밝은 부분은 더 밝게
- 어두운 부분은 더 어둡게
- 전체적으로 더 선명한 이미지가 됨

마치 TV의 화면 설정에서 '선명도'나 '대비'를 조절하는 것처럼, 우리도 이미지를 더 선명하게 만들어 식물의 상태를 더 잘 관찰할 수 있습니다.





In [ ]:
def enhance_contrast(image):
    """이미지 대비 향상 함수"""
    # LAB 색공간으로 변환
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    # CLAHE로 대비 향상
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    l = clahe.apply(l)

    # 다시 합치기
    enhanced = cv2.merge((l,a,b))
    enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2RGB)

    return enhanced

# 대비 향상 적용
enhanced_img = enhance_contrast(original_img)

# 결과 시각화
plt.figure(figsize=(12, 5))

plt.subplot(121)
plt.imshow(original_img)
plt.title('Original Image')
plt.axis('off')

plt.subplot(122)
plt.imshow(enhanced_img)
plt.title('Enhanced Image')
plt.axis('off')

plt.tight_layout()
plt.show()

print("\n=== 이미지 개선 완료 ===")
print("- 대비가 향상되어 식물의 특징이 더 선명하게 보입니다.")
print("- 이후 분석을 위한 전처리가 완료되었습니다.")

# 3. 식생 지수(NDVI) 계산

식물의 건강 상태를 숫자로 나타내는 방법이 있습니다. 마치 우리가 체온계로 열이 있는지 확인하는 것처럼, NDVI는 식물의 '건강 체온계' 역할을 합니다.

실생활에서 볼 수 있는 비슷한 예시를 들어보면:
- 열화상 카메라로 건물의 단열 상태를 확인하는 것
- 이마에 대는 체온계로 발열 체크하는 것
- 의사가 청진기로 심장 소리를 듣는 것

NDVI는 식물이 빛을 반사하는 정도를 측정해서 건강 상태를 판단합니다:
- 건강한 식물일수록 더 많은 빛을 반사
- 스트레스를 받은 식물은 반사율이 낮음
- 죽은 식물이나 마른 잎은 거의 반사하지 않음

이것은 마치 운동장에서 흰 운동화를 신은 학생들을 보는 것과 비슷합니다:
- 새 운동화는 빛을 잘 반사해서 하얗게 보임
- 오래 신은 운동화는 반사율이 떨어져서 누렇게 변함
- 매우 오래된 운동화는 거의 반사하지 않아 어둡게 보임

In [ ]:
def calculate_ndvi(image):
   """NoIR 카메라를 위한 NDVI 계산 함수"""
   # 이미지를 각각의 색상 채널로 분리
   b, g, r = cv2.split(image)

   # 계산을 위해 float 타입으로 변환
   bottom = (r.astype(float) + b.astype(float))
   bottom[bottom == 0] = 0.01  # 0으로 나누는 것을 방지

   # NoIR 카메라용 NDVI 계산 (Blue 필터 특성 반영)
   ndvi = (r.astype(float) - b) / bottom

   return ndvi

# NDVI 계산하기
ndvi = calculate_ndvi(enhanced_img)

# 결과 시각화
plt.figure(figsize=(12, 5))

plt.subplot(121)
plt.imshow(enhanced_img)
plt.title('Enhanced Plant Image')
plt.axis('off')

plt.subplot(122)
ndvi_plot = plt.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
plt.colorbar(ndvi_plot, label='NDVI Value')
plt.title('NDVI Map')
plt.axis('off')

plt.tight_layout()
plt.show()

# NDVI 통계 출력
print("\n=== NDVI 분석 결과 ===")
print(f"최소값: {np.min(ndvi):.3f}")
print(f"최대값: {np.max(ndvi):.3f}")
print(f"평균값: {np.mean(ndvi):.3f}")
print("\n값의 의미:")
print("- 1에 가까울수록 식물이 건강한 상태")
print("- 0에 가까울수록 보통의 상태")
print("- -1에 가까울수록 건강하지 않은 상태")

# 4. 식생 지수 컬러 매핑

우리는 지금까지 계산한 NDVI 값을 더 알기 쉽게 색으로 표현해보겠습니다.

## 컬러 매핑의 이해
일상생활에서 색으로 정보를 표현하는 예시를 보면:
- 교통 신호등: 빨간불(위험), 노란불(주의), 초록불(안전)
- 태풍 경로도: 파란색(약한 비), 노란색(강한 비), 빨간색(매우 강한 비)
- 체온계: 파란색(저온), 초록색(정상), 빨간색(고온)

## 식물 건강 상태의 색상 표현
마찬가지로 식물의 건강 상태도 색으로 표현할 수 있습니다:
- 진한 초록색: 매우 건강한 상태 (광합성이 활발함)
- 연한 초록색: 건강한 상태
- 노란색: 보통 상태
- 주황색: 약간의 스트레스 상태
- 빨간색: 건강하지 않은 상태

## 필요한 파일 준비
이 분석을 위해서는 특별한 색상 매핑 파일이 필요합니다:
1. '[fastiecm.py](https://drive.google.com/file/d/1QEe_PCI1eKOvKdGGXIkS_GjARgQBgDYq/view?usp=drive_link)' 파일을 현재 작업 폴더에 업로드해야 합니다
2. 이 파일에는 식생 분석에 최적화된 색상 정보가 담겨 있습니다
3. 파일이 없으면 코드가 실행되지 않으니 반드시 준비해주세요

이렇게 색으로 표현하면 식물의 어느 부분이 건강한지, 어느 부분이 관리가 필요한지 한눈에 알 수 있습니다.



In [ ]:
# 4. 컬러 매핑 셀

# fastiecm.py 파일 업로드
from google.colab import files
print("fastiecm.py 파일을 업로드하세요.")
uploaded = files.upload()

# fastiecm 임포트
from fastiecm import fastiecm

def contrast_stretch(im):
    """시각화를 위한 대비 스트레칭 함수"""
    # 상위/하위 5% 제외한 범위 사용
    in_min = np.percentile(im, 5)
    in_max = np.percentile(im, 95)

    out_min = 0
    out_max = 255

    # 값 범위 조정
    out = im - in_min
    out *= ((out_min - out_max) / (in_min - in_max))
    out += in_min

    return out

try:
    # NDVI 대비 향상
    ndvi_contrasted = contrast_stretch(ndvi)

    # 컬러 매핑 적용
    color_mapped = cv2.applyColorMap(ndvi_contrasted.astype(np.uint8), fastiecm)

    # 결과 시각화
    plt.figure(figsize=(12, 5))

    plt.subplot(121)
    plt.imshow(cv2.cvtColor(color_mapped, cv2.COLOR_BGR2RGB))
    plt.title('Color Mapped NDVI')
    plt.axis('off')

    plt.subplot(122)
    ndvi_plot = plt.imshow(ndvi, cmap='RdYlGn')
    plt.colorbar(ndvi_plot, label='NDVI Value')
    plt.title('NDVI Distribution')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # 식물 건강 상태 분포 출력
    print("\n=== 식물 건강 상태 분포 ===")
    total_pixels = ndvi.size
    healthy = np.sum((ndvi > 0.3))
    moderate = np.sum((ndvi >= 0) & (ndvi <= 0.3))
    stressed = np.sum(ndvi < 0)

    print(f"건강한 상태 (진한 초록색): {healthy/total_pixels*100:.1f}%")
    print(f"보통 상태 (연한 초록색~노란색): {moderate/total_pixels*100:.1f}%")
    print(f"관리 필요 상태 (주황색~빨간색): {stressed/total_pixels*100:.1f}%")

except Exception as e:
    print("\n=== 오류 발생 ===")
    print("fastiecm.py 파일이 올바르게 업로드되지 않았습니다.")
    print("파일을 다시 업로드하고 실행해주세요.")
    print(f"오류 메시지: {str(e)}")

# NDVI 결과를 활용한 인공지능 실습 사례

## 1. 식물 건강 상태 분류하기
- NDVI 값을 기준으로 식물의 건강 상태를 분류하는 모델 만들기
- 입력: NDVI 이미지
- 출력: '건강', '보통', '관리 필요' 등급
- 활용 모델: 이미지 분류 CNN (예: ResNet, VGG16)

## 2. 생장 예측하기
- 일주일 간격으로 찍은 NDVI 데이터로 다음 주 생장 상태 예측
- 입력: 3-4주간의 연속적인 NDVI 데이터
- 출력: 다음 주의 예상 NDVI 값
- 활용 모델: LSTM, RNN과 같은 시계열 예측 모델

## 3. 잎의 면적 측정하기
- NDVI 이미지에서 건강한 잎의 면적을 자동으로 계산
- 입력: NDVI 이미지
- 출력: 잎의 실제 면적 (cm² 단위)
- 활용 모델: 객체 탐지 모델 (예: YOLO, Mask R-CNN)

## 4. 비료 필요 영역 찾기
- NDVI 값이 낮은 부분을 자동으로 검출하여 관리가 필요한 영역 표시
- 입력: NDVI 이미지
- 출력: 관리가 필요한 영역의 위치와 면적
- 활용 모델: 영역 분할(Segmentation) 모델